# 📝 Data Collection Final
---

This is where the main data collection happens. 

In [1]:
# install IGDB API wrapper
%pip install igdb-api-v4

Note: you may need to restart the kernel to use updated packages.


In [2]:
# imports
from igdb.wrapper import IGDBWrapper
from config import *
import json
import pandas as pd

# initialize wrapper
wrapper = IGDBWrapper(CLIENT_ID, ACCESS_TOKEN)

In [24]:
# iterative API request using wrapper
min_id = 0
max_id = 500
limit = 500
start = 200000

vg_dataset = []
vg_counts = []

for i in range(start, start+50000, limit):
    byte_array = wrapper.api_request(
        'games',
        f'fields name, summary, genres, genres.name; limit {limit}; where id > {min_id+i} & id <= {max_id+i} & summary != null & genres != null; sort id asc;'
    )

    json_array = json.loads(byte_array)
    arr_size = len(json_array)

    vg_dataset += json_array
    vg_counts.append(arr_size)

In [25]:
videogame_df = pd.DataFrame(vg_dataset)

videogame_df['genre_count'] = [len(x) for x in videogame_df['genres']]
videogame_df['genre_list'] = [[y['name'] for y in x] for x in videogame_df['genres']]
videogame_df['genre_ids'] = [[y['id'] for y in x] for x in videogame_df['genres']]

videogame_df = videogame_df.drop("genres", axis='columns').sort_index(ascending=True)
print(vg_counts, sum(vg_counts))

[273, 310, 373, 355, 343, 358, 361, 410, 406, 369, 391, 257, 167, 213, 369, 373, 200, 202, 175, 162, 234, 327, 406, 451, 323, 336, 348, 329, 288, 323, 316, 406, 253, 466, 467, 237, 181, 300, 342, 340, 213, 333, 320, 315, 109, 289, 423, 236, 276, 442, 347, 326, 424, 415, 265, 233, 266, 301, 339, 340, 276, 282, 342, 255, 348, 386, 296, 318, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 21471


In [26]:
videogame_df.to_csv(f'./final_data2/data{start}_s{sum(vg_counts)}.csv', index=False)